# Setup

In [1]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import probplot

SEED = 42
random.seed(SEED)
np.random.seed(SEED)

base_df = pd.read_csv("data_1.csv", index_col=0)

/tmp/ipykernel_14493/2922023937.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Univariate outlier detection according to the rule of thumb:
# Values that lie outside the interval [E(Salary)-3,5*σ, E(Salary)+3,5*σ] are considered outliers
mean_salary = base_df['salary'].mean()
std_salary = base_df['salary'].std()

lower_bound = mean_salary - 3.5 * std_salary
upper_bound = mean_salary + 3.5 * std_salary

print(f"Lower bound: {lower_bound}")
print(f"Upper bound: {upper_bound}")

outliers = base_df[(base_df['salary'] < lower_bound) | (base_df['salary'] > upper_bound)]
outliers.sort_values(by='salary')

Lower bound: -33673.691524622074
Upper bound: 268921.08149783587


,score,salary,happiness,Age
797,88,-164801.150081,1.0,48
372,77,-151112.222803,1.0,78
801,22,-92822.767823,1.0,78
484,56,-84455.351609,20.0,54
364,44,-69725.546360,20.0,71
51,36,-48194.031763,1.0,30
528,77,-46857.508922,1.0,23
773,52,-38788.531785,20.0,22
883,49,269075.046479,81.0,40
991,42,297158.567626,81.0,79


According to our domain knowledge about salary (non-negative, outliers for large values likely), we can find different outliers:

In [3]:
outliers = base_df[(base_df['salary'] < 0)]
outliers.sort_values(by='salary')

,score,salary,happiness,Age
797,88,-164801.150081,1.0,48
372,77,-151112.222803,1.0,78
801,22,-92822.767823,1.0,78
484,56,-84455.351609,20.0,54
364,44,-69725.546360,20.0,71
51,36,-48194.031763,1.0,30
528,77,-46857.508922,1.0,23
773,52,-38788.531785,20.0,22
348,59,-31883.990479,20.0,65
271,97,-20946.363596,1.0,38
